# Exploration des techniques de clustering

Le but de ce tp est de faire face à la problématique: 
<center style="color:red" >  Voici XXX documents -bruts, non étiquetés-... Comment les valoriser? Les exploiter? Les comprendre? Les résumer? </center>

Nous avons vu dans les séances précédentes comment représenter les données textuelles sous forme de sacs de mots:
$$X = 
	\begin{matrix} 
	 & \textbf{t}_j \\
	 & \downarrow \\
	\textbf{d}_i \rightarrow &
	\begin{pmatrix} 
	x_{1,1} & \dots & x_{1,D} \\
	\vdots & \ddots & \vdots \\
	x_{N,1} & \dots & x_{N,D} \\
	\end{pmatrix}
	\end{matrix}
	$$

A partir de cette représentation, les questions qui se posent sont les suivantes:
1. Quel algorithme de clustiering choisir?
    - K-means, LSA, pLSA, LDA
1. Quels résultats attendre?
    - qualité, bruit, exploitabilité immédiate etc...
1. Quelles analyses qualitatives effectuer pour comprendre les groupes?
1. Comment boucler, itérer pour améliorer la qualité du processus?


> <span style="color:magenta" > La tâche est difficile, dans ce TP, on part d'un **jeu de données étiquetées** afin de pouvoir appliquer des métriques quantitatives sur les résultats obtenus. </span>


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import codecs
import re
import os.path
import sklearn

## Chargement des données



In [3]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [28]:
# conversion BoW + tf-idf

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer() # TfidfVectorizer(max_features=500)
vectors = vectorizer.fit_transform(newsgroups_train.data)
print(vectors.shape)

# mesure de la sparsité = 157 mots actif par document sur 130000 !!
print(vectors.nnz / float(vectors.shape[0]))

(11314, 500)
70.5516174650875


In [29]:
# retrouver les mots
print([(i,vectorizer.get_feature_names()[i]) \
       for i in np.random.randint(vectors.shape[1], size=10)])

[(54, 'anything'), (269, 'maybe'), (100, 'case'), (132, 'doesn'), (152, 'everything'), (46, 'american'), (287, 'nasa'), (444, 'type'), (66, 'available'), (384, 'show')]


In [30]:
# gestion des étiquettes (pour l'évaluation seulemnet en non-supervisé)
Y = newsgroups_train.target
print(Y[:10]) # numérique
print([newsgroups_train.target_names[i] for i in Y[:10]]) # vraie classe

[ 7  4  4  1 14 16 13  3  2  4]
['rec.autos', 'comp.sys.mac.hardware', 'comp.sys.mac.hardware', 'comp.graphics', 'sci.space', 'talk.politics.guns', 'sci.med', 'comp.sys.ibm.pc.hardware', 'comp.os.ms-windows.misc', 'comp.sys.mac.hardware']


# Tests préliminaires

Commençons par le commencement: tout problème non-supervisé (ou presque) doit être analysé en premier lieu avec les $k$-means!


In [31]:
from sklearn.cluster import KMeans
# Algo => risque de prendre du temps si le vocabulaire n'a pas été réduit !!
# Note: on dirait que l'algo transforme les données en dense vector=> catastrophe pour nous !!!
# => limitation du nombre d'itération arbitraire + limitation du vocabulaire

kmeans = KMeans(n_clusters=20, random_state=0, max_iter=10).fit(vectors)

In [ ]:
# analyse

# recupération des proto:
kmeans.cluster_centers_

# mots les plus importants par cluster => TODO
# version print / version word cloud

### Limites

- Limites liées à la description
    - trop de mots
    - trop de mots fréquents qui déroutent l'algorithme
    - ...
- Limites liées à l'algorithme
    - distance euclidienne absurde

Les limites algorithmiques vont être résolues en changeant d'algorithme... Les limites de représentation des données seront résolues par votre capacité en ingénierie.


Algorithmes à tester:
- LSA
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD
- LDA
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

**Note:** pour des tests rapides, il est plus simple de rester dans le cadre de scikit-learn... Néanmoins, dans un milieu industriel, il faudrait exploiter des outils plus efficaces comme ceux présents dans la librairie ```gensim```. Si vous vous sentez à l'aise avec la donnée textuelles, allez directement vers ces outils:
https://radimrehurek.com/gensim/models/ldamodel.html

## Evaluation des performances

Les performances sont très dures à évaluer en clustering... Ce qui explique que cette évaluation est souvent, au moins partiellement, qualitative (=étudiée à l'oeil, sur des exemples ou des paramètres). 
Afin d'éviter de faire n'importe quoi, il faut aussi réfléchir à des métriques quantitatives.

### Qualitatif

Analyser le vocabulaire des différents clusters
1. En terme de mots les plus fréquents, les plus probables ou de dimensions des vecteurs propres les plus fortes.
1. En terme de mots discriminants
    - construction de critère de contraste (type odd's ratio) entre la présence dans une classe et présence dans les autres classe
1. Affichage 
    - des 10 ou 20 mots critiques de chaque classe ```print```
    - sous la forme de word cloud
    - affichage interactif avancé: http://www.kennyshirley.com/LDAvis/
        - pour une version intégrable dans un notebook: https://github.com/bmabey/pyLDAvis
        - merci de l'utiliser **après avoir compris le principe de réduction de la dimensionalité pour les clusters**
    
### Quantitatif

Pour donner des chiffres, il faut des étiquettes. C'est rarement le cas sur des jeux de données industriels... Mais c'est bon dans un cadre académique comme 20 newsgroups!

**Problème:** Comme nos algorithmes sont non supervisés, les sorties (bien que catégorielles) ne sont pas alignées avec l'encodage des étiquettes du jeu de données. Il faut trouver des astuces.

1. Etude basique sur la taille des clusters
    - est ce qu'une classe n'a pas tout attrapé?
1. Pureté = extraction de la classe majoritaire dans un cluster + calcul de la pureté du cluster
    - 1 score par cluster par défaut
    - agrégation par somme pondérée sur la taille des clusters
1. Indice de Rand  https://fr.wikipedia.org/wiki/Indice_de_Rand
1. Métrique adaptée à une hypothèse spécifique


## Vers une version plus évoluée des algorithmes

1. Si l'un des clusters attiré toutes les données: êtes-vous capable de supprimer ce cluster et de simplement répartir les échantillons dans les autres catégories?

1. Si vous avez une idée vague des thématiques que vous souhaitez voir isolées... 
    - trouver 10 mots dans chaque catégories
    - biaiser l'initialisation pour attirer ces classes

1. Si vous mettez un utilisateur dans la boucle
    - passer en mode supervisé multiclasse et exploiter les feedbacks de l'utilisateur pour forcer le passage d'un document dans la classe d'à coté 
        - Naive Bayes, SVM ou autre...
    - réfléchir à une approche active qui sélectionne les documents les plus intéressants à montrer à l'utilisateur pour lui demander un étiquetage.